In [ ]:
import tushare as ts
ts.inst_tops(days = 10) #近10交易日内机构席位追踪列表

# 返回值说明：

# code:代码
# name:名称
# bamount:累积买入额(万)
# bcount:买入次数
# samount:累积卖出额(万)
# scount:卖出次数
# net:净额(万)

In [ ]:
stock_quote = '300142'#改变股票代码即可得到结果值
data = ts.get_hist_data(stock_quote)
data.to_csv('c:/ASHARES/temp.csv') #注意保存路径必须要与这个"Notebook" 在相同的文件夹中. 


In [ ]:
#搭建HMM模型
import datetime
import numpy as np
import pandas as pd
from matplotlib import cm, pyplot as plt
from hmmlearn.hmm import GaussianHMM
df = pd.read_csv("temp.csv", header=0)
df.head()
print("Data_size：", df.shape) 
print("Data_headers", df.columns)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.reset_index(inplace=True, drop=False)
df.drop(['index','open','high','low','price_change','p_change', 'ma5', 'ma10', 'ma20', 'v_ma5', 'v_ma10', 'v_ma20'], axis=1, inplace=True)
df['date'] = df['date'].apply(datetime.datetime.toordinal)
print(df.head())
dates = df['date'][1:]
close_v = df['close']
volume = df['volume'][1:]
diff = np.diff(close_v)

X = np.column_stack([diff, volume])
print("New_data_Size：", X.shape)

In [ ]:
min = X.mean(axis=0)[0] - 8*X.std(axis=0)[0]
max = X.mean(axis=0)[0] + 8*X.std(axis=0)[0] 
X = pd.DataFrame(X)
for i in range(len(X)):
    if (X.loc[i, 0]< min) | (X.loc[i, 0] > max):
            X.loc[i, 0] = X.mean(axis=0)[0]

In [ ]:
X_Test = X.iloc[:-30]
X_Pre = X.iloc[-30:]
print("traning_size：", X_Test.shape)     
print("test_size：", X_Pre.shape)    

model = GaussianHMM(n_components=3, covariance_type='diag', n_iter=1000) 
model.fit(X_Test)

# print(model.transmat_) 
#状态转移矩阵 - 状态转移矩阵：代表三个隐藏层状态的转移概率。可以看出对角线的数值较大，即状态0、1、2都倾向保持当前的状态，意味该股票较稳。

# print(model.means_) 
#均值矩阵 - 共三行，每一行代表一种隐藏层状态（状态0、1、2），每一行的两个元素分别代表涨幅值的均值和成交量的均值。
# 状态0均值为负值，可以解释为“跌”；状态1均值最小，接近0，可以解释为“平”，状态2均值为正，可以解释为“涨”。

# print(model.covars_) 
#协方差矩阵 - 协方差矩阵：共三个协方差矩阵，分别对应三种隐藏层状态。对角线的值为该状态下的方差，方差越大，代表该状态的预测不可信。

# print(model.n_components)  #隐藏状态的个数

In [ ]:
expected_returns_volumes = np.dot(model.transmat_, model.means_)
expected_returns = expected_returns_volumes[:,0]        
predicted_price = []  
current_price = close_v.iloc[-30]
for i in range(len(X_Pre)):
    hidden_states = model.predict(X_Pre.iloc[i].values.reshape(1,2))  
    predicted_price.append(current_price+expected_returns[hidden_states])
    current_price = predicted_price[i]

x = dates[-29: ]
y_act = close_v[-29:]
y_pre = pd.Series(predicted_price[:-1])
plt.figure(figsize=(8,6))
plt.plot_date(x, y_act,linestyle="-",marker="None",color='g')
plt.plot_date(x, y_pre,linestyle="-",marker="None",color='r')
plt.legend(['Actual', 'Predicted'])
plt.show()